In [1]:
class Const:  # an overriding descriptor, see later
    def __init__(self, value):
        self.value = value

    def __set__(self, *_):  # silently ignore any attempt at setting
        pass

    def __get__(self, *_):  # always return the constant value
        return self.value

    def __delete__(self, *_):  # const cannot be deleted
        pass


class X:
    c = Const(23)


In [2]:
x = X()
print(x.c)

23


In [3]:
x.c = 42
print(x.c)

23


In [4]:
del x.c
print(x.c)

23


In [5]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info('Accessing %r giving %r', 'age', value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', 'age', value)
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1                   # Calls both __get__() and __set__()

In [6]:
mary = Person('Mary M', 30) 

INFO:root:Updating 'age' to 30


In [7]:
dave = Person('David D', 40)

INFO:root:Updating 'age' to 40


In [8]:
vars(mary)

{'name': 'Mary M', '_age': 30}

In [9]:
vars(dave)

{'name': 'David D', '_age': 40}

In [10]:
mary.age

INFO:root:Accessing 'age' giving 30


30

In [11]:
dave.name

'David D'

In [12]:
dave.age

INFO:root:Accessing 'age' giving 40


40

In [13]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        value = getattr(obj, self.private_name)
        logging.info('Accessing %r giving %r', self.public_name, value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', self.public_name, value)
        setattr(obj, self.private_name, value)

class Person:

    name = LoggedAccess()                # First descriptor instance
    age = LoggedAccess()                 # Second descriptor instance

    def __init__(self, name, age):
        self.name = name                 # Calls the first descriptor
        self.age = age                   # Calls the second descriptor

    def birthday(self):
        self.age += 1

In [14]:
vars(vars(Person)['name'])

{'public_name': 'name', 'private_name': '_name'}

In [15]:
pete = Person('Peter P', 10)

INFO:root:Updating 'name' to 'Peter P'
INFO:root:Updating 'age' to 10


In [16]:
kate = Person('Catherine C', 20)

INFO:root:Updating 'name' to 'Catherine C'
INFO:root:Updating 'age' to 20


In [17]:
vars(pete)

{'_name': 'Peter P', '_age': 10}

In [18]:
vars(kate)

{'_name': 'Catherine C', '_age': 20}

In [19]:
from abc import ABC, abstractmethod

class Validator(ABC):

    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass

In [20]:
class OneOf(Validator):

    def __init__(self, *options):
        self.options = set(options)

    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {value!r} to be one of {self.options!r}')

class Number(Validator):

    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue is not None and value < self.minvalue:
            raise ValueError(
                f'Expected {value!r} to be at least {self.minvalue!r}'
            )
        if self.maxvalue is not None and value > self.maxvalue:
            raise ValueError(
                f'Expected {value!r} to be no more than {self.maxvalue!r}'
            )

class String(Validator):

    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate

    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f'Expected {value!r} to be an str')
        if self.minsize is not None and len(value) < self.minsize:
            raise ValueError(
                f'Expected {value!r} to be no smaller than {self.minsize!r}'
            )
        if self.maxsize is not None and len(value) > self.maxsize:
            raise ValueError(
                f'Expected {value!r} to be no bigger than {self.maxsize!r}'
            )
        if self.predicate is not None and not self.predicate(value):
            raise ValueError(
                f'Expected {self.predicate} to be true for {value!r}'
            )

In [21]:
class Component:

    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf('wood', 'metal', 'plastic')
    quantity = Number(minvalue=0)

    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity

In [22]:
Component('Widget', 'metal', 5)

ValueError: Expected <method 'isupper' of 'str' objects> to be true for 'Widget'

In [23]:
Component('WIDGET', 'metle', 5)

ValueError: Expected 'metle' to be one of {'wood', 'plastic', 'metal'}

In [24]:
Component('WIDGET', 'metal', -5)

ValueError: Expected -5 to be at least 0

In [25]:
Component('WIDGET', 'metal', 'V')

TypeError: Expected 'V' to be an int or float

In [27]:
class C6:
    def __init__(self, n): self.x = n

inst = C6(42)

In [28]:
inst.x

42

In [29]:
class C7:
    pass
z = C7()
z.x = 23
print(z.x)

23


In [30]:
print(z.__class__.__name__, z.__dict__)

C7 {'x': 23}


In [31]:
z.y = 45
z.__dict__['z'] = 67
print(z.x, z.y, z.z)

23 45 67


In [32]:
class SpecialCase:
    def amethod(self):
        print('special')
class NormalCase:
    def amethod(self):
        print('normal')
def appropriate_case(isnormal=True):
    if isnormal:
        return NormalCase()
    else:
        return SpecialCase()
aninstance = appropriate_case(isnormal=False)
aninstance.amethod() 

special


In [33]:
class Singleton:
    _singletons = {}
    def __new__(cls, *args, **kwds):
        if cls not in cls._singletons:
            cls._singletons[cls] = obj = super().__new__(cls)
            obj._initialized = False
        return cls._singletons[cls]

In [34]:
class S1(Singleton):pass

In [42]:
s1 = S1()
s2 = S1()
s1._initialized

False

In [43]:
print(s1, s2)

<__main__.S1 object at 0x7fe0d0f22530> <__main__.S1 object at 0x7fe0d0f22530>


In [44]:
s1 is s2

True

In [45]:
Singleton._singletons

{__main__.S1: <__main__.S1 at 0x7fe0d0f22530>}

In [46]:
class B:
    a = 23
    b = 45
    def f(self):
        print('method f in class B')
    def g(self):
        print('method g in class B')
class C(B):
    b = 67
    c = 89
    d = 123
    def g(self):
        print('method g in class C')
    def h(self):
        print('method h in class C')
x = C()
x.d = 77
x.e = 88

In [50]:
print(x.a, x.b, x.c, x.d, x.e)

23 67 89 77 88


In [51]:
x.f(); x.g(); x.h()

method f in class B
method g in class C
method h in class C


In [52]:
x.f = lambda x: x*x

In [53]:
x.f(4)

16

In [54]:
x.__class__

__main__.C

In [56]:
C.__bases__

(__main__.B,)

In [57]:
x.__class__.__bases__

(__main__.B,)

In [58]:
C.__dict__

mappingproxy({'__module__': '__main__',
              'b': 67,
              'c': 89,
              'd': 123,
              'g': <function __main__.C.g(self)>,
              'h': <function __main__.C.h(self)>,
              '__doc__': None})

In [59]:
x.__dict__

{'d': 77, 'e': 88, 'f': <function __main__.<lambda>(x)>}

In [61]:
print(x.h, x.g, x.f, C.h, C.g, C.f)

<bound method C.h of <__main__.C object at 0x7fe0d0f3b520>> <bound method C.g of <__main__.C object at 0x7fe0d0f3b520>> <function <lambda> at 0x7fe0d0b68dc0> <function C.h at 0x7fe0d0b6a8c0> <function C.g at 0x7fe0d0b6acb0> <function B.f at 0x7fe0d0b6af80>


In [62]:
x.h()

method h in class C


In [63]:
class B:
    a = 23
    b = 45
    def f(self):
        print('method f in class B')
    def g(self):
        print('method g in class B')
class C(B):
    b = 67
    c = 89
    d = 123
    def g(self):
        print('method g in class C')
    def h(self):
        print('method h in class C')

In [64]:
x = C()
x.g()

method g in class C


In [65]:
x.b

67

In [66]:
x.a

23

In [67]:
class Base:
    def greet(self, name):
        print('Welcome', name)
class Sub(Base):
    def greet(self, name):
        print('Well Met and', end=' ')
        Base.greet(self, name)
x = Sub()
x.greet('Alex')

Well Met and Welcome Alex


In [68]:
class A:
    def met(self):
        print('A.met')
class B(A):
    def met(self):
        print('B.met')
        A.met(self)
class C(A):
    def met(self):
        print('C.met')
        A.met(self)
class D(B,C):
    def met(self):
        print('D.met')
        B.met(self)
        C.met(self)

In [69]:
class A:
    def met(self):
        print('A.met')
class B(A):
    def met(self):
        print('B.met')
        super().met()
class C(A):
    def met(self):
        print('C.met')
        super().met()
class D(B,C):
    def met(self):
        print('D.met')
        super().met()

In [73]:
class AClass:
    def astatic():
        print('a static method')
    astatic = staticmethod(astatic)


class ABase:
    def aclassmet(cls):
        print('a class method for', cls.__name__)
    aclassmet = classmethod(aclassmet)
class ADeriv(ABase):
    pass


In [75]:
print(AClass.astatic())
print(ABase.aclassmet())

a static method
None
a class method for ABase
None


In [76]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    def area(self):
        return self.width * self.height
    area = property(area, doc='area of the rectangle')

In [77]:
r = Rectangle(20,40)
r.area

800

In [78]:
r.area()

TypeError: 'int' object is not callable

In [79]:
r.__dict__

{'width': 20, 'height': 40}

In [81]:
import math
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    @property
    def area(self):
        """area of the rectangle"""
        return self.width * self.height
    @area.setter
    def area(self, value):
        scale = math.sqrt(value/self.area)
        self.width *= scale
        self.height *= scale

In [82]:
r = Rectangle(20,40)
r.area

800

In [83]:
r.area = 600

In [84]:
r.width, r.height

(17.32050807568877, 34.64101615137754)

In [85]:
class B:
    def f(self):
        return 23
    g = property(f)
class C(B):
    def f(self):
        return 42

c = C()
print(c.g)               

23


In [86]:
class B:
    def f(self):
        return 23
    def _f_getter(self):
        return self.f()
    g = property(_f_getter)
class C(B):
    def f(self):
        return 42

c = C()
print(c.g)

42


In [87]:
class listNoAppend(list):
    def __getattribute__(self, name):
        if name == 'append':
            raise AttributeError(name)
        return list.__getattribute__(self, name)

In [88]:
def fake_get_item(idx):
    return idx
class MyClass:
    pass
n = MyClass()
n.__getitem__ = fake_get_item
print(n[23])

TypeError: 'MyClass' object is not subscriptable

In [89]:
import enum
import stat

class Permission(enum.Flag):
    EXEC_OTH = stat.S_IXOTH
    WRITE_OTH = stat.S_IWOTH
    READ_OTH = stat.S_IROTH
    EXEC_GRP = stat.S_IXGRP
    WRITE_GRP = stat.S_IWGRP
    READ_GRP = stat.S_IRGRP
    EXEC_USR = stat.S_IXUSR
    WRITE_USR = stat.S_IWUSR
    READ_USR = stat.S_IRUSR

    @classmethod
    def from_stat(cls, stat_result):
        return cls(stat_result.st_mode & 0o777)

from pathlib import Path

cur_dir = Path.cwd()
dir_perm = Permission.from_stat(cur_dir.stat())
if dir_perm & Permission.READ_OTH:
    print(f'{cur_dir} is readable by users outside the owner group')

# the following raises TypeError: Flag enums do not support order 
# comparisons
print(Permission.READ_USR > Permission.READ_OTH)

/home/richard/Dev/pynut4/04_Object_Oriented_Python is readable by users outside the owner group


TypeError: '>' not supported between instances of 'Permission' and 'Permission'